# Non supervised text topic detection

## LDA with Gensim

In [1]:
import os
import re
import string

#import pandas as pd

data_path = '.'

In [2]:
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
print(len(twenty_train['data']))

11314


In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer

def tokenize_doc(text):
    sents = sent_tokenize(text)
    tokens = []
    for s in sents:
        for t in TreebankWordTokenizer().tokenize(s):
            if len(t)<2 or '\n' in t or '\t' in t or t.isnumeric():
                continue
            else:
                tokens += [t.lower()]
    return tokens

In [4]:
%%time
import nltk
nltk.download('punkt')

docs_tokenized = []
for text in twenty_train['data']:
    docs_tokenized += [tokenize_doc(text)]
print(len(docs_tokenized))

[nltk_data] Downloading package punkt to /Users/jorge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


11314
CPU times: user 23.9 s, sys: 122 ms, total: 24 s
Wall time: 24.2 s


In [5]:
# Get the NLTK stopwords list
import nltk

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

In [7]:
# create dictionary and bag of words corpus

from gensim.corpora import Dictionary, MmCorpus
nltk.download('stopwords')
print('Building dictionary...')
dictionary = Dictionary(docs_tokenized)

# Stopwords

additional_stopwords=set(['...', '\'s','\'\'',  "``", "n\'t", "\'re", "\'m", "\'ve",
                          '--', '"', "\'ax", 'max', 'q,3'])

#additional_stopwords=set()

stopwords = nltk_stopwords().union(additional_stopwords)
stopword_ids = map(dictionary.token2id.get, stopwords)
dictionary.filter_tokens(stopword_ids)
dictionary.compactify()

# Filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.95, keep_n=None)
dictionary.compactify()

print('Building corpus...')
corpus = [dictionary.doc2bow(doc) for doc in docs_tokenized]

[nltk_data] Downloading package stopwords to /Users/jorge/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Building dictionary...
Building corpus...


In [8]:
%%time

# Train the LDA model
from gensim import models

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=25, passes=10)
                                      
lda.save(os.path.join(data_path, 'newsgroups_50_lda.model'))


CPU times: user 3min 40s, sys: 24.8 s, total: 4min 5s
Wall time: 1min 3s


In [9]:
# Print summary of topics
lda.show_topics()


[(18,
  '0.017*"would" + 0.015*"one" + 0.012*"people" + 0.009*"think" + 0.007*"like" + 0.007*"even" + 0.006*"say" + 0.006*"many" + 0.006*"see" + 0.005*"know"'),
 (24,
  '0.034*"db" + 0.017*"\\/" + 0.016*"m/" + 0.016*"q\\" + 0.015*"mv" + 0.010*"sq" + 0.009*"p\\" + 0.009*"m6" + 0.008*"/-" + 0.008*"m+"'),
 (16,
  '0.020*"thanks" + 0.020*"would" + 0.017*"anyone" + 0.016*"know" + 0.014*"please" + 0.013*"like" + 0.011*"one" + 0.011*"get" + 0.011*"problem" + 0.009*"need"'),
 (2,
  '0.022*"list" + 0.010*"faq" + 0.009*"send" + 0.009*"_/" + 0.008*"min" + 0.006*"posted" + 0.006*"group" + 0.006*"random" + 0.006*"posting" + 0.006*"mail"'),
 (10,
  '0.026*"soon" + 0.021*"gordon" + 0.020*"quality" + 0.019*"motorola" + 0.019*"banks" + 0.018*"surrender" + 0.018*"marriage" + 0.016*"skepticism" + 0.015*"shameful" + 0.015*"intellect"'),
 (6,
  '0.033*"file" + 0.029*"entry" + 0.027*"*/" + 0.025*"/*" + 0.015*"output" + 0.014*"entries" + 0.013*"program" + 0.013*"section" + 0.012*"rules" + 0.010*"int"'),
 (5,

In [10]:
# get_document_topics
lda.get_document_topics(corpus[:10])


In [11]:
# Print topics details
for i in range(lda.num_topics):
    print(i, lda.show_topic(i))
    print('\n')
    

0 [('b8f', 0.034056496), ('a86', 0.028083092), ('1d9', 0.021552382), ('pl+', 0.016177818), ('0t-', 0.010828815), ('7ey', 0.01075693), ('giz', 0.010474111), ('/3t', 0.009324538), ("'as", 0.008282298), (',3', 0.007772687)]


1 [('mac', 0.016375238), ('system', 0.014542106), ('memory', 0.013801628), ('bus', 0.011037328), ('apple', 0.010419637), ('speed', 0.010084982), ('ram', 0.009396631), ('data', 0.008245424), ('keyboard', 0.0077940994), ('machine', 0.00779364)]


2 [('list', 0.022212967), ('faq', 0.009787526), ('send', 0.009478541), ('_/', 0.00890162), ('min', 0.007608283), ('posted', 0.0062425067), ('group', 0.0061428696), ('random', 0.00594738), ('posting', 0.0059285676), ('mail', 0.005835992)]


3 [('drive', 0.045417838), ('disk', 0.017504357), ('hard', 0.016568689), ('drives', 0.0156257), ('sale', 0.014944168), ('price', 0.014706716), ('controller', 0.012591627), ('scsi', 0.012544425), ('new', 0.012431246), ('offer', 0.011125698)]


4 [('1st', 0.011929442), ('..', 0.011810667), ('c

# Visualization

In [12]:
# Install pyLDAvis in colab

! pip install pyldavis


In [13]:
import pyLDAvis

pyLDAvis.enable_notebook()


In [14]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)


/Users/jorge/anaconda3/envs/bankia/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     16.390102        1       1 -0.213166  0.063122
21     11.480266        1       2 -0.194158  0.068297
15      9.529619        1       3 -0.122922 -0.135726
22      8.698516        1       4 -0.111144  0.134639
16      5.979698        1       5 -0.186722 -0.140901
19      5.211185        1       6 -0.174307 -0.080599
23      4.785672        1       7 -0.102807  0.162115
8       4.534439        1       8 -0.056186  0.037492
5       3.584846        1       9  0.055866 -0.030766
12      3.517549        1      10 -0.007710  0.263047
11      3.407440        1      11 -0.005552  0.032407
0       2.704413        1      12  0.306554 -0.054861
17      2.661955        1      13 -0.091060 -0.039018
1       2.590824        1      14 -0.132704 -0.155760
3       2.365329        1      15  0.003804 -0.201811
2       2.112971        1      16  0.028424 -0.050090
7       1.814516        1      17 -0.004361  0.204987
14      1.763080        1      18  0.000219  0.034317
6       1.747343        1      19  0.034922 -0.090836
9       1.322962        1      20  0.235549  0.020951
4       1.109862        1      21  0.059837 -0.052094
24      1.012862        1      22  0.264169 -0.038472
20      0.922679        1      23  0.026953 -0.022376
10      0.511611        1      24  0.173871  0.004244
13      0.240261        1      25  0.212633  0.067692, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
9168   Default  2225.000000           god  2225.000000  30.0000  30.0000
17352  Default  1559.000000          file  1559.000000  29.0000  29.0000
6467   Default  1133.000000         drive  1133.000000  28.0000  28.0000
11902  Default  1130.000000           key  1130.000000  27.0000  27.0000
14893  Default  6597.000000         would  6597.000000  26.0000  26.0000
14686  Default  3501.000000          know  3501.000000  25.0000  25.0000
7783   Default  1090.000000         space  1090.000000  24.0000  24.0000
6448   Default   909.000000           b8f   909.000000  23.0000  23.0000
17130  Default  1251.000000        thanks  1251.000000  22.0000  22.0000
14723  Default  4131.000000        people  4131.000000  21.0000  21.0000
14961  Default  3270.000000           use  3270.000000  20.0000  20.0000
13569  Default  1394.000000       program  1394.000000  19.0000  19.0000
14831  Default   908.000000         jesus   908.000000  18.0000  18.0000
14991  Default   750.000000           a86   750.000000  17.0000  17.0000
12354  Default  1898.000000        system  1898.000000  16.0000  16.0000
979    Default  1641.000000        anyone  1641.000000  15.0000  15.0000
8915   Default  1806.000000          said  1806.000000  14.0000  14.0000
15101  Default  1205.000000    government  1205.000000  13.0000  13.0000
488    Default   840.000000          team   840.000000  12.0000  12.0000
3026   Default  1230.000000          data  1230.000000  11.0000  11.0000
1199   Default   914.000000          list   914.000000  10.0000  10.0000
4668   Default  4021.000000          like  4021.000000   9.0000   9.0000
845    Default   798.000000          game   798.000000   8.0000   8.0000
10280  Default  3609.000000           get  3609.000000   7.0000   7.0000
7833   Default  2603.000000           new  2603.000000   6.0000   6.0000
4704   Default   547.000000        israel   547.000000   5.0000   5.0000
9871   Default   576.000000           1d9   576.000000   4.0000   4.0000
2119   Default   710.000000          chip   710.000000   3.0000   3.0000
12269  Default  1308.000000          year  1308.000000   2.0000   2.0000
10295  Default   511.000000         entry   511.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
5604   Topic25    14.658722      recieved    15.658847   5.9652  -5.0859
16039  Topic25   